In [ ]:
import pandas as pd

df = pd.read_excel("C:\\Users\\fernando.araujo_logg\\Downloads\\Payables.xlsx")

# Passo 1: Agrupa pelas duas colunas e conta as ocorrências
duplicatas = df.groupby(['Loggi ID', 'Tipo Documento']).size().reset_index(name='Contagem')

# Passo 2: Filtra onde a contagem é maior que 1
duplicatas = duplicatas[duplicatas['Contagem'] > 1]

# Passo 3 (opcional): Recupera as linhas originais que fazem parte dessas duplicatas
df_duplicados = df.merge(duplicatas[['Loggi ID', 'Tipo Documento']], on=['Loggi ID', 'Tipo Documento'], how='inner')

# Exibe o resultado
# display(df_duplicados)

df_duplicados.to_excel("C:\\Users\\fernando.araujo_logg\\Downloads\\duplicidades.xlsx", index=False)

In [ ]:
import requests
import google.generativeai as genai
from time import sleep as slp
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
import pandas as pd
from pathlib import Path

# Carrega .env
env_path = Path('auth/.env')
load_dotenv(dotenv_path=env_path)

# Configura chave da API do Gemini
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("API KEY do Gemini não encontrada no .env")

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.0-flash')

# Datas de busca
dtinicio = '20250701'
dtfim = '20250716'

# URL da API
url = f"https://loggi-prod.it-cpi008-rt.cfapps.br10.hana.ondemand.com/http/v1/read/drivtransf?DATE_START={dtinicio}&DATE_END={dtfim}"

# Autenticação da API
username = os.getenv("API_USERNAME")
password = os.getenv("API_PASSWORD")
if not username or not password:
    raise ValueError("Usuário ou senha da API não encontrados no .env")

# Chamada da API
response = requests.get(url, auth=HTTPBasicAuth(username, password))

if response.status_code == 200:
    dados_json = response.json()

    # Transforma em DataFrame
    df = pd.DataFrame(dados_json)

    print("Colunas disponíveis:", df.columns.tolist())  # DEBUG

    # Define colunas desejadas e filtra apenas as que existem
    colunas_desejadas = ['LOGGI_ID', 'DOCTYPE', 'BP_NUMBER', 'AMOUT', 'PERIOD_START', 'PERIOD_END', 'MSG']
    colunas_existentes = [col for col in colunas_desejadas if col in df.columns]

    if not colunas_existentes:
        raise ValueError("Nenhuma das colunas esperadas foi encontrada na resposta da API.")

    df_filtrado = df[colunas_existentes]

    # Geração de sugestões via Gemini
    sugestoes = []

    for i, msg in enumerate(df_filtrado.get('MSG', [])):
        try:
            if not msg:
                sugestoes.append("Mensagem vazia.")
                continue

            resposta = model.generate_content(msg)
            sugestoes.append(resposta.text)
            slp(1)  # delay para evitar rate limit
        except Exception as e:
            sugestoes.append(f"Erro ao processar: {str(e)}")

    df_filtrado['sugestao_gemini'] = sugestoes

    print("\nResultado com sugestões:")
    print(df_filtrado)

else:
    print(f"Erro ao consultar API: {response.status_code}")
    print(response.text)


In [ ]:
import pandas as pd

# Carregar o CSV
df = pd.read_csv('members.csv', sep=';', encoding='utf-8')

# Filtrar colunas
df = df[['Name', 'Role', 'Last login']]

# Converter 'Last login' para data (sem hora)
df['Last login'] = pd.to_datetime(df['Last login']).dt.date

# Calcular a diferença entre hoje e o último login (em dias)
df['Dias desde o último login'] = pd.to_datetime('today').date() - df['Last login']

# Filtrar membros que não logaram nos últimos 30 dias
df = df[df['Dias desde o último login'] > pd.Timedelta(days=30)]


display(df) # Exibir o resultado

,Name,Role,Last login,Dias desde o último login
1,Jonatas Santana (LEX Jhow),company_guest,2024-04-15,"473 days, 0:00:00"
5,julio cezar tosti rocha,company_guest,2024-05-02,"456 days, 0:00:00"
9,Luciana Ribeiro de Souza Santos,company_guest,2025-05-05,"88 days, 0:00:00"
13,Kaique,company_guest,2024-02-28,"520 days, 0:00:00"
20,Luana Carvalho da Silva,company_guest,2024-03-14,"505 days, 0:00:00"
27,Andre Aloise,company_guest,2024-01-08,"571 days, 0:00:00"
29,Nicolas Brasiliano Gonçalves Carrão,company_guest,2024-06-24,"403 days, 0:00:00"
30,Raphael Castilho Gil,company_guest,2025-04-09,"114 days, 0:00:00"
32,Joelma Bueno,company_guest,2025-05-30,"63 days, 0:00:00"
36,Nicolas de Matos Silva,company_guest,2024-04-05,"483 days, 0:00:00"
